# Lecture 1 Demo: Introduction to Quantum Machine Learning

Computational companion notebook for Lecture 1.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram

import pennylane as qml

print(f"Qiskit version: {__import__('qiskit').__version__}")
print(f"PennyLane version: {qml.__version__}")

## Demo 1. First quantum circuit in Qiskit

A single qubit, one Hadamard gate, measurement. Every qubit starts in $|0\rangle$ by default (this is a universal convention in circuit-model quantum computing).

$H|0\rangle = |+\rangle = (|0\rangle + |1\rangle)/\sqrt{2}$, so we expect roughly 50/50 outcomes.

In [ ]:
qc = QuantumCircuit(1)
qc.h(0)
qc.measure_all()

print(qc.draw("text"))

In [ ]:
sampler = StatevectorSampler()
result = sampler.run([qc], shots=1024).result()
counts = result[0].data.meas.get_counts()

print(f"Counts: {counts}")
plot_histogram(counts)

## Demo 2. Same circuit in PennyLane

PennyLane uses a functional interface. By default, `default.qubit` returns exact probabilities (no shot noise).

In [ ]:
dev = qml.device("default.qubit", wires=1)


@qml.qnode(dev)
def hadamard_circuit():
    qml.Hadamard(wires=0)
    return qml.probs(wires=0)


probs = hadamard_circuit()
print(f"P(|0>) = {probs[0]:.4f},  P(|1>) = {probs[1]:.4f}")
print(qml.draw(hadamard_circuit)())

In [ ]:
# With shot noise (as on real hardware)
dev_shots = qml.device("default.qubit", wires=1, shots=1024)


@qml.qnode(dev_shots)
def hadamard_shots():
    qml.Hadamard(wires=0)
    return qml.counts()


counts_pl = hadamard_shots()
print(f"PennyLane counts (1024 shots): {counts_pl}")

## Demo 3. Bloch sphere visualization

The six cardinal states: $|0\rangle, |1\rangle, |+\rangle, |-\rangle, |i\rangle, |-i\rangle$.

In [ ]:
def statevector_to_bloch(sv):
    """Convert a 2-element statevector to Bloch coordinates (x, y, z)."""
    a, b = sv[0], sv[1]
    bx = 2 * np.real(a * np.conj(b))
    by = 2 * np.imag(a * np.conj(b))
    bz = np.abs(a)**2 - np.abs(b)**2
    return np.array([bx, by, bz])


state_defs = {
    "|0>": [],
    "|1>": [("x", 0)],
    "|+>": [("h", 0)],
    "|->": [("x", 0), ("h", 0)],
    "|i>": [("h", 0), ("s", 0)],
    "|-i>": [("h", 0), ("sdg", 0)],
}

bloch_vectors = {}
for name, gates in state_defs.items():
    qc_tmp = QuantumCircuit(1)
    for gate_name, qubit in gates:
        getattr(qc_tmp, gate_name)(qubit)
    sv = Statevector.from_instruction(qc_tmp)
    bloch_vectors[name] = statevector_to_bloch(sv.data)
    print(f"{name:5s}  Bloch = ({bloch_vectors[name][0]:+.2f}, "
          f"{bloch_vectors[name][1]:+.2f}, {bloch_vectors[name][2]:+.2f})")

In [ ]:
u = np.linspace(0, 2 * np.pi, 30)
v = np.linspace(0, np.pi, 20)
x_sphere = np.outer(np.cos(u), np.sin(v))
y_sphere = np.outer(np.sin(u), np.sin(v))
z_sphere = np.outer(np.ones(np.size(u)), np.cos(v))

fig, axes = plt.subplots(
    2, 3, figsize=(15, 10), subplot_kw={"projection": "3d"}
)

for idx, (name, vec) in enumerate(bloch_vectors.items()):
    ax = axes.flat[idx]
    ax.plot_wireframe(
        x_sphere, y_sphere, z_sphere, alpha=0.08, color="gray"
    )
    for s in (-1.2, 1.2):
        ax.plot([s, -s], [0, 0], [0, 0], "k-", alpha=0.15)
        ax.plot([0, 0], [s, -s], [0, 0], "k-", alpha=0.15)
        ax.plot([0, 0], [0, 0], [s, -s], "k-", alpha=0.15)
    ax.quiver(
        0, 0, 0, vec[0], vec[1], vec[2],
        color="red", arrow_length_ratio=0.12, linewidth=2.5,
    )
    ax.scatter([vec[0]], [vec[1]], [vec[2]], color="red", s=80)
    ax.set_title(name, fontsize=14)
    ax.set_xlim(-1.2, 1.2)
    ax.set_ylim(-1.2, 1.2)
    ax.set_zlim(-1.2, 1.2)
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")

plt.suptitle("Six cardinal states on the Bloch sphere", fontsize=15)
plt.tight_layout()
plt.show()

## Demo 4. Phase matters: $|+\rangle$ vs $|-\rangle$

Both give 50/50 in the $Z$-basis. Applying another Hadamard (measuring in the $X$-basis) distinguishes them perfectly.

In [ ]:
# Prepare |+> then measure in X-basis (H before measurement)
qc_plus = QuantumCircuit(1)
qc_plus.h(0)     # prepare |+>
qc_plus.h(0)     # rotate to X-basis
qc_plus.measure_all()

# Prepare |-> then measure in X-basis
qc_minus = QuantumCircuit(1)
qc_minus.x(0)
qc_minus.h(0)    # prepare |->
qc_minus.h(0)    # rotate to X-basis
qc_minus.measure_all()

sampler = StatevectorSampler()
results = sampler.run([qc_plus, qc_minus], shots=1024).result()

counts_plus = results[0].data.meas.get_counts()
counts_minus = results[1].data.meas.get_counts()

print(f"|+> measured in X-basis: {counts_plus}")
print(f"|-> measured in X-basis: {counts_minus}")
print()
print("Same 50/50 in Z-basis, but perfectly distinguishable in X-basis.")
print("The relative phase is what makes the difference.")

## Demo 5. Parameterized rotation $R_y(\theta)$

$R_y(\theta)|0\rangle = \cos(\theta/2)|0\rangle + \sin(\theta/2)|1\rangle$. This is the building block of variational quantum circuits.

In [ ]:
dev_exact = qml.device("default.qubit", wires=1)


@qml.qnode(dev_exact)
def ry_probs(theta):
    qml.RY(theta, wires=0)
    return qml.probs(wires=0)


thetas = np.linspace(0, 2 * np.pi, 100)
probs_sweep = np.array([ry_probs(t) for t in thetas])

fig, ax = plt.subplots(figsize=(9, 4))
ax.plot(thetas, probs_sweep[:, 0], "b-", lw=2, label=r"$P(|0\rangle)$")
ax.plot(thetas, probs_sweep[:, 1], "r-", lw=2, label=r"$P(|1\rangle)$")
ax.set_xlabel(r"$\theta$ (rad)")
ax.set_ylabel("Probability")
ax.set_title(r"Outcome probabilities for $R_y(\theta)|0\rangle$")
ax.legend()
ax.set_xticks([0, np.pi / 2, np.pi, 3 * np.pi / 2, 2 * np.pi])
ax.set_xticklabels(["0", r"$\pi/2$", r"$\pi$", r"$3\pi/2$", r"$2\pi$"])
ax.grid(True, alpha=0.3)
ax.axhline(y=0.5, color="gray", ls="--", alpha=0.5)
plt.tight_layout()
plt.show()